In [ ]:
import ROOT 
import tensorflow as tf
import numpy as np
import pandas
import matplotlib.pyplot as plt
import math as mt
import os
#from tqdm import tqdm
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon, Circle, Rectangle
from matplotlib.backends.backend_agg import FigureCanvasAgg
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import random
from mpl_toolkits.axes_grid1 import make_axes_locatable

from scipy import stats
import matplotlib
import mplhep as hep
#hep.set_style(hep.style.ROOT)
hep.set_style("CMS")
#hep.set_style("ALICE")
#hep.set_style("firamath")
hep.set_style({"font.sans-serif":'Tex Gyre Heros'})

In [ ]:
def mkdir(path_name):
    try:
        os.mkdir(path_name)
    except:
        print("folder already there")

# Preprare Training 

In [ ]:
def transform(x, means, sigma):
    for e_id in range(len(x)):
        for j_id in range(len(x[e_id])):
            x[e_id][j_id] = (x[e_id][j_id] - means) / sigma
            
            
    return x


def transform_back(x, means, sigma):
    for e_id in range(len(x)):
        for j_id in range(len(x[e_id])):
            x[e_id][j_id] = x[e_id][j_id]* sigma + means
              
    return x

In [ ]:
from copy import deepcopy
def Standardize(sam1):
    
    sam = deepcopy(sam1)
    jets = []
    for i in sam:
        for j in i:
            jets.append(j)
            
    jets = np.array(jets)
    
    means = jets.mean(axis=0)
    sigma = jets.std(axis=0)
    
    for e_id in range(len(sam)):
        for j_id in range(len(sam[e_id])):
            sam[e_id][j_id] = (sam[e_id][j_id] - means) / sigma
            
    
    return sam, means, sigma

In [ ]:
def generate_weights(sample):
    weights = []
    
    for event in sample:
        
        sum_ = 0.1
        if len(event) > 2:
            sum_ += event[0][3] + event[1][3]
        else:
            sum_ += event[0][3]
        
        
        weights.append(np.array(sum_))
            
    return np.array(weights)

# Training 

In [ ]:
from tensorflow.keras.models import Sequential
from keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.layers import Input,Concatenate, Conv1D, GlobalMaxPooling2D, Conv2D, Lambda, Layer, Reshape, LSTM, Permute, Dense,GlobalMaxPooling1D, MaxPooling1D,MaxPooling2D, TimeDistributed, Dropout, Bidirectional, BatchNormalization, Flatten
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
import keras.backend as K
import tensorflow.keras.layers as L
from tensorflow.keras.models import model_from_json

In [ ]:
class SpatialPyramidPooling(Layer):
    '''Spatial pyramid pooling layer for 2D inputs.
    See Spatial Pyramid Pooling in Deep Convolutional Networks for Visual Recognition,
    K. He, X. Zhang, S. Ren, J. Sun
    # Arguments
        pool_list: list of int
            List of pooling regions to use. The length of the list is the number of pooling regions,
            each int in the list is the number of regions in that pool. For example [1,2,4] would be 3
            regions with 1, 2x2 and 4x4 max pools, so 21 outputs per feature map
    # Input shape
        4D tensor with shape:
        `(samples, channels, rows, cols)` if dim_ordering='th'
        or 4D tensor with shape:
        `(samples, rows, cols, channels)` if dim_ordering='tf'.
    # Output shape
        2D tensor with shape:
        `(samples, channels * sum([i * i for i in pool_list])`
    '''

    def __init__(self, pool_list, **kwargs):

        self.dim_ordering = K.image_dim_ordering()
        assert self.dim_ordering in {'tf', 'th'}, 'dim_ordering must be in {tf, th}'

        self.pool_list = pool_list

        self.num_outputs_per_channel = sum([i * i for i in pool_list])

        super(SpatialPyramidPooling, self).__init__(**kwargs)

    def build(self, input_shape):
        if self.dim_ordering == 'th':
            self.nb_channels = input_shape[1]
        elif self.dim_ordering == 'tf':
            self.nb_channels = input_shape[3]

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], self.nb_channels * self.num_outputs_per_channel)

    def get_config(self):
        config = {'pool_list': self.pool_list}
        base_config = super(SpatialPyramidPooling, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def call(self, x, mask=None):

        input_shape = K.shape(x)

        if self.dim_ordering == 'th':
            num_rows = input_shape[2]
            num_cols = input_shape[3]
        elif self.dim_ordering == 'tf':
            num_rows = input_shape[1]
            num_cols = input_shape[2]

        row_length = [K.cast(num_rows, 'float32') / i for i in self.pool_list]
        col_length = [K.cast(num_cols, 'float32') / i for i in self.pool_list]

        outputs = []

        if self.dim_ordering == 'th':
            for pool_num, num_pool_regions in enumerate(self.pool_list):
                for jy in range(num_pool_regions):
                    for ix in range(num_pool_regions):
                        x1 = ix * col_length[pool_num]
                        x2 = ix * col_length[pool_num] + col_length[pool_num]
                        y1 = jy * row_length[pool_num]
                        y2 = jy * row_length[pool_num] + row_length[pool_num]

                        x1 = K.cast(K.round(x1), 'int32')
                        x2 = K.cast(K.round(x2), 'int32')
                        y1 = K.cast(K.round(y1), 'int32')
                        y2 = K.cast(K.round(y2), 'int32')
                        new_shape = [input_shape[0], input_shape[1],
                                     y2 - y1, x2 - x1]
                        x_crop = x[:, :, y1:y2, x1:x2]
                        xm = K.reshape(x_crop, new_shape)
                        pooled_val = K.max(xm, axis=(2, 3))
                        outputs.append(pooled_val)

        elif self.dim_ordering == 'tf':
            for pool_num, num_pool_regions in enumerate(self.pool_list):
                for jy in range(num_pool_regions):
                    for ix in range(num_pool_regions):
                        x1 = ix * col_length[pool_num]
                        x2 = ix * col_length[pool_num] + col_length[pool_num]
                        y1 = jy * row_length[pool_num]
                        y2 = jy * row_length[pool_num] + row_length[pool_num]

                        x1 = K.cast(K.round(x1), 'int32')
                        x2 = K.cast(K.round(x2), 'int32')
                        y1 = K.cast(K.round(y1), 'int32')
                        y2 = K.cast(K.round(y2), 'int32')

                        new_shape = [input_shape[0], y2 - y1,
                                     x2 - x1, input_shape[3]]

                        x_crop = x[:, y1:y2, x1:x2, :]
                        xm = K.reshape(x_crop, new_shape)
                        pooled_val = K.max(xm, axis=(1, 2))
                        outputs.append(pooled_val)

        if self.dim_ordering == 'th':
            outputs = K.concatenate(outputs)
        elif self.dim_ordering == 'tf':
            # outputs = K.concatenate(outputs,axis = 1)
            outputs = K.concatenate(outputs)
            # outputs = K.reshape(outputs,(len(self.pool_list),self.num_outputs_per_channel,input_shape[0],input_shape[1]))
            # outputs = K.permute_dimensions(outputs,(3,1,0,2))
            outputs = K.reshape(outputs,(input_shape[0], self.num_outputs_per_channel * self.nb_channels))

        return outputs

In [ ]:
def generate_data():
    """Replaces Keras' native ImageDataGenerator."""
    i = -1
    while True:
        i += 1  
        if i == len(x_train): i = 0
        
        yield x_train[i], y_train[i], np.array([train_w[i]])

In [ ]:
def generate_val():
    """Replaces Keras' native ImageDataGenerator."""
    i = -1
    while True:
        i += 1  
        if i == len(x_test): i = 0
        #print(x_test[i], y_test[i])
        #print(x_test[i].shape, y_test[i].shape)
        
        yield x_test[i], y_test[i]

In [ ]:
def LSTM_CNN():
    
    inputs = tf.keras.Input(shape=( None, 6))
    
    #extract features with 1D-CNN.. None refers to the variable number of jets per event
    
    conv_layer = Conv1D(64, 2, strides=1, padding="same", activation='relu')(inputs) # output shape (None, 3, 32)
    conv_layer_2 = Conv1D(128, 3, strides=1, padding="same", activation='relu')(conv_layer) # output shape (None, 1, 64)
    
    pool = GlobalMaxPooling1D(data_format='channels_first')(conv_layer_2) # output shape (None, 1)
    
    
    res = tf.expand_dims(pool, axis=2) #make "N" (None) timestamps each of one variable
    
    
    lstm = LSTM(50, return_sequences=True, dropout=0.2, kernel_initializer=tf.keras.initializers.glorot_normal(seed=777), bias_initializer='zeros')
    lstm = Bidirectional(lstm)(res)
    
    #Attention mechanism before output
    
    attention=TimeDistributed(Dense(1, activation = 'tanh'))(lstm)
    attention=L.Softmax(axis=1)(attention)
    context=L.Multiply()([attention,lstm])
    cout=L.Lambda(lambda x: K.sum(x,axis=1))(context)
    
    #Sigmoid activated output
    out = Dense(1, activation='sigmoid')(cout)
    
    model = Model(inputs, outputs=[out])
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.00002), metrics=[tf.keras.metrics.PrecisionAtRecall(0.85, num_thresholds=100)])
    
    
    return model

In [ ]:
model = LSTM_CNN()
model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, min_delta=0.0001)
history = model.fit(generate_data(), steps_per_epoch=len(x_train), epochs=100, callbacks = [callback],class_weight={0:4, 1:1}, validation_data=generate_val(), validation_steps=len(x_test))

# Save Model

In [ ]:
##### mkdir("Pretrained")

model_json = model.to_json()
with open("Pretrained/CNNLSTM_CaloJetsPt30.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("Pretrained/CNNLSTM_CaloJetsPt30.h5")

# Load Model

In [ ]:
from tensorflow.keras.models import Sequential
from keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.layers import Input,Concatenate, Conv1D, GlobalMaxPooling2D, Conv2D, Lambda, Layer, Reshape, LSTM, Permute, Dense,GlobalMaxPooling1D, MaxPooling1D,MaxPooling2D, TimeDistributed, Dropout, Bidirectional, BatchNormalization, Flatten
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
import keras.backend as K
import tensorflow.keras.layers as L
from tensorflow.keras.models import model_from_json
import numpy as np

In [ ]:
#2D-CNN

json_file = open('Pretrained/CNN2DLSTM_CaloJetsPt30.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("Pretrained/CNN2DLSTM_CaloJetsPt30.h5")
print("Loaded model from disk")

# Plot the model

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='1D_LSTM_CNN_ptsort.pdf', show_shapes=True, show_layer_names=True)

# ROC CURVE

In [ ]:
from copy import deepcopy
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

y_pred_model = deepcopy(y_pred)

y_pred_model = np.array([i[0][0] for i in y_pred_model])

fpr_keras_dnn, tpr_keras_dnn, thresholds_keras_dnn = roc_curve(y_test, y_pred_model)
auc_keras_dnn = auc(fpr_keras_dnn, tpr_keras_dnn)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras_dnn, tpr_keras_dnn, 'r', label='auc_keras_dnn  (area = {:.3f})'.format(auc_keras_dnn))
plt.legend(loc='best')

# Plotting Attention

In [ ]:
import matplotlib

from scipy import stats
import matplotlib
import mplhep as hep
#hep.set_style(hep.style.ROOT)
hep.set_style("CMS")
#hep.set_style("ALICE")
#hep.set_style("firamath")
hep.set_style({"font.sans-serif":'Tex Gyre Heros'})

In [ ]:
def show_attention(sam, val, y, p):
    
    
    fig, ax = plt.subplots(1,1, figsize=(10, 10), dpi = 160)
    plot = []

    for j in sam:
        #plot.append([j]*6)
        plot.append([j]*4)
        
    
    val = val[::-1]

    #im = ax.imshow(plot,cmap=plt.get_cmap('Purples'), alpha=0.7, extent = [0, 6, 0, len(plot)])
    im = ax.imshow(plot,cmap=plt.get_cmap('Purples'), alpha=0.7, extent = [0, 4, 0, len(plot)])
    #ax.set_xticklabels([r"$p_T$", r"$\eta$", r"$\phi$", r"$btag$", r"$m_j$", r"$E$"])
    ax.set_xticklabels([r"$p_T$", r"$\eta$", r"$\phi$", r"$btag$"])
    #xlabel = [r"$p_T$", r"$\eta$", r"$\phi$", r"$btag$", r"$m_j$", r"$E$"]
    xlabel = [r"$p_T$", r"$\eta$", r"$\phi$", r"$btag$"]
    #ax.set_xticks(np.arange(0.5, 6.5,1))
    ax.set_xticks(np.arange(0.5, 4.5,1))
    ax.set_yticks(np.arange(0.5,len(plot)+0.5,1))
    # ... and label them with the respective list entries
    ax.set_xticklabels(xlabel)
    ax.set_yticklabels(np.arange(len(plot)))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cb = fig.colorbar(im, fraction=0.046, pad=0.04, cax=cax)
    
    cb.ax.set_yticklabels([ round(i, 2) for i in np.arange(min(sam), max(sam), (max(sam)-min(sam))/10)], fontsize=20)
    
    ax.tick_params(axis='both', which='major', labelsize=20)
    ax.tick_params(axis='both', which='minor', labelsize=20)
    
    half = 0.8*max(sam)
    
    print(half, max(sam))
    
    
    for i in range(len(val)):
        #for j in range(0,6):
        for j in range(0,4):
            #text = ax.text(j+0.5, i+0.5, "%.2f" % val[i, j],
                           #ha="center", va="center", color="black", size= 15)
                
            if sam[::-1][i] < half:
                text = ax.text(j+0.5, i+0.5, "%.2f" % val[i][j],
                               ha="center", va="center", color="black", size= 20)
            else:
                print(sam[::-1][i], half)
                text = ax.text(j+0.5, i+0.5, "%.2f" % val[i][j],
                               ha="center", va="center", color="white", size= 20)
                
    if y == 0:
        t = "Class QCD Background"
    else:
        t = r"Class $HH\rightarrow 4b$"
        
    t += r"  $P ( C=HH\rightarrow 4b | X )$ = " + "%.3f" % p[0][0]
    
    ax.text(0, len(plot) + 0.07, t, size=20)
    
    fig.tight_layout()
    
    return fig

In [ ]:
# Here it gets the output of a llayer called "softmax". These are the attention weifghts we want to plot
grad_model = Model([model.inputs], 
                   [model.get_layer("softmax").output])


# divide attention score for signal and bkg events
atts_sig = []
preds_sig = []


# extract 10 signal pred
j = 0
for i in range(1000):
    
    if y_test[i] != 1: continue
    else:
        
        # use grad_model to get the attention score for this event
        atts_sig.append(grad_model.predict(x_test[i])[0])
        # use full model to get the posterior prob.
        preds_sig.append(model.predict(x_test[i]))
        j+= 1
    # just save 10 events
    if j == 10: break


print("done")

# Do the same for bkg

atts_bkg = []
preds_bkg = []
j = 0
for i in range(1000):
    if y_test[i] != 0: continue
    else:
        atts_bkg.append(grad_model.predict(x_test[i])[0])
        preds_bkg.append(model.predict(x_test[i]))
        j+=1
    if j == 10: break
        
print("done")      
atts = []
preds = []



# collect 100000 of them 

#for i in range(len(x_test)):
for i in range(10000):
    
    if i % 100 == 0: print(float(i) / 10000 * 100)
    
    
    atts.append(grad_model.predict(x_test[i])[0])
    preds.append(model.predict(x_test[i]))


In [ ]:
# Reshape attention scores in order to plot the color map with imshow

c = []

for i in range(len(atts)):
    n = atts[i].reshape(atts[i].shape[0])
    c.append(n)

# c is the new array of attention scores
c = np.array(c)

In [ ]:
# Plot attention score overlayd with the event image
# only for 100 events
# and only for bkg events (y_test == 0)
# and only if the model predicts a signal class
# Meaning this examples are false positives

j = 0
i = 0
while 1:
    if preds[i] > 0.85 and y_test[i] == 0:
        fig = show_attention(c[i], X_test[i][0], y_test[i], preds[i])
        fig.savefig("Attention/high/QCD_{}.pdf".format(i))
        j+=1 
    i+= 1
    if j == 100: break